In [2]:
# 1. Imports & Config
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, cross_validate
import joblib

# use your pre‑processed v2 dataset (already contains y_plus_*h)
DATA_PATH    = r"C:\Users\Linds\Repos\East_River\data\training\east_river_training-v2.h5"
HORIZONS     = [24, 48, 72]
TS_CV        = TimeSeriesSplit(n_splits=3)
MODEL_PARAMS = dict(tree_method='hist', random_state=0)

# 2. Load preprocessed data
def load_data(path):
    # key='df' matches how you saved v2
    return pd.read_hdf(path, key='df')

# 3. Define features & targets (no shifting needed)
def prepare(df, horizons):
    # drop leakage & existing targets
    drop_cols = [
        'local_time',
        'last_control_time',
        'OnLine_Load_MW',
        'Load_Control_MW',
        'Control_Threshold_MW'
    ] + [f'y_plus_{h}h' for h in horizons]
    feature_cols = [c for c in df.columns if c not in drop_cols]
    # drop location if present
    X = df[feature_cols].drop(columns=['location'], errors='ignore')
    # map horizons to their y vectors
    y = {h: df[f'y_plus_{h}h'] for h in horizons}
    return X, y

# 4. Train/Eval routine
def run_pipeline():
    df = load_data(DATA_PATH)
    X, y_map = prepare(df, HORIZONS)
    split = int(0.8 * len(X))

    results = []
    for h in HORIZONS:
        y = y_map[h]
        X_tr, X_ho = X.iloc[:split], X.iloc[split:]
        y_tr, y_ho = y.iloc[:split], y.iloc[split:]

        # CV on train slice
        cv = cross_validate(
            XGBRegressor(**MODEL_PARAMS),
            X_tr, y_tr,
            cv=TS_CV,
            scoring=['neg_mean_absolute_error','neg_root_mean_squared_error'],
            n_jobs=1
        )

        # final fit & hold‑out eval
        model = XGBRegressor(**MODEL_PARAMS)
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_ho)

        mae  = mean_absolute_error(y_ho, y_pred)
        rmse = mean_squared_error(y_ho, y_pred, squared=False)

        # persist
        joblib.dump(model, f"xgb_v2_{h}h.pkl")

        results.append({
            'horizon':  h,
            'cv_mae':  -cv['test_neg_mean_absolute_error'].mean(),
            'cv_rmse': -cv['test_neg_root_mean_squared_error'].mean(),
            'ho_mae':   mae,
            'ho_rmse':  rmse
        })

    return pd.DataFrame(results).set_index('horizon')

if __name__ == "__main__":
    df_res = run_pipeline()
    print(df_res)

c:\Users\Linds\anaconda3\envs\load_forecasting_env\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Linds\anaconda3\envs\load_forecasting_env\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


            cv_mae    cv_rmse     ho_mae    ho_rmse
horizon                                            
24       11.752477  16.830566   9.544760  13.032488
48       11.938297  16.732316  10.007958  13.590096
72       12.362068  17.479692  10.359988  13.970080


c:\Users\Linds\anaconda3\envs\load_forecasting_env\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [1]:
# 1. Imports & Config
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, cross_validate
import joblib

DATA_PATH = r'C:\Users\Linds\Repos\East_River\data\training\east_river_processed_dataset-v4.h5'
HORIZONS = [24, 48, 72]
TS_CV = TimeSeriesSplit(n_splits=3)
MODEL_PARAMS = dict(tree_method='hist', random_state=0)

# 2. Load & preprocess
def load_and_prep(path):
    df = pd.read_hdf(path)
    # date features & encode
    df['dow_sin'] = np.sin(2*np.pi*df.day_of_week_num/7)
    df['dow_cos'] = np.cos(2*np.pi*df.day_of_week_num/7)
    df.sort_values('local_time', inplace=True)
    df.reset_index(drop=True, inplace=True)
    # bool→int & categorical codes
    bools = df.select_dtypes(include='bool').columns
    df[bools] = df[bools].astype(int)
    for c in df.select_dtypes(exclude=[np.number]).columns.difference(['location','local_time']):
        df[c] = df[c].astype('category').cat.codes
    return df

df = load_and_prep(DATA_PATH)

# 3. Create shifted targets & drop leakage
for H in HORIZONS:
    df[f'y_plus_{H}h'] = df.OnLine_Load_MW.shift(-H)
df.dropna(subset=[f'y_plus_{H}h' for H in HORIZONS], inplace=True)
leak = ['local_time','last_control_time','OnLine_Load_MW','Load_Control_MW','Control_Threshold_MW']
leak += [f'y_plus_{H}h' for H in HORIZONS]
FEATURES = [c for c in df.columns if c not in leak]

# 4. Train/Eval routine
results = []
for H in HORIZONS:
    X = df[FEATURES].drop(columns='location')
    y = df[f'y_plus_{H}h']
    # hold‑out split
    split = int(0.8*len(X))
    X_tr, X_ho = X.iloc[:split], X.iloc[split:]
    y_tr, y_ho = y.iloc[:split], y.iloc[split:]
    # CV
    cv = cross_validate(XGBRegressor(**MODEL_PARAMS),
                        X_tr, y_tr,
                        cv=TS_CV,
                        scoring=['neg_mean_absolute_error','neg_root_mean_squared_error'],
                        n_jobs=1)
    # retrain & hold‑out
    m = XGBRegressor(**MODEL_PARAMS)
    m.fit(X_tr, y_tr)
    y_pred = m.predict(X_ho)
    mae, rmse = mean_absolute_error(y_ho, y_pred), mean_squared_error(y_ho, y_pred, squared=False)
    joblib.dump(m, f'xgb_final_{H}h.pkl')
    results.append({
      'h':H,
      'CV_MAE': -cv['test_neg_mean_absolute_error'].mean(),
      'CV_RMSE': -cv['test_neg_root_mean_squared_error'].mean(),
      'HO_MAE': mae,
      'HO_RMSE': rmse
    })

pd.DataFrame(results).set_index('h')

c:\Users\Linds\anaconda3\envs\load_forecasting_env\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Linds\anaconda3\envs\load_forecasting_env\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Linds\anaconda3\envs\load_forecasting_env\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


,CV_MAE,CV_RMSE,HO_MAE,HO_RMSE
h,,,,
24,11.752477,16.830566,9.544760,13.032488
48,11.938297,16.732316,10.007958,13.590096
72,12.362068,17.479692,10.359988,13.970080


In [ ]:
HO_MAE and HO_RMSE are the hold‑out (HO) set error metrics you compute after you train your model on the training slice and predict on the held‑out slice:

• HO_MAE – mean absolute error on the hold‑out data:  
    mae = mean_absolute_error(y_ho, y_pred)  

• HO_RMSE – root mean squared error on the hold‑out data:  
    rmse = mean_squared_error(y_ho, y_pred, squared=False)

In [ ]:
Here’s what’s going on:

• CV (cross‑validation)  
    – You take your training slice (first 80% of X/y), split it into e.g. 3 time‑series folds (`TS_CV`)  
    – For each fold you train on two sub‑folds and score on the third  
    – You then average those fold scores to get `cv_mae` and `cv_rmse`  

• HO (hold‑out)  
    – You train a fresh model on the entire training slice (first 80%)  
    – You predict on the reserved hold‑out slice (last 20%) and compute MAE/RMSE there  
    – That gives you `ho_mae` and `ho_rmse`  

The .pkl files  
    – Are the trained XGBRegressor objects (one per horizon) serialized via `joblib.dump`  
    – You can later load them with `joblib.load("xgb_v2_24h.pkl")` (for example) to  
        • Inspect the model  
        • Make new predictions on unseen data  
        • Deploy without retraining from scratch

In [ ]:
# 6. Detailed evaluation & visualization
import matplotlib.pyplot as plt

# reload data & split
df     = load_data(DATA_PATH)
X, y_map = prepare(df, HORIZONS)
split  = int(0.8 * len(X))
X_ho   = X.iloc[split:]

fig, axes = plt.subplots(len(HORIZONS), 3, figsize=(15, 5 * len(HORIZONS)))

for i, h in enumerate(HORIZONS):
    model   = joblib.load(f"xgb_v2_{h}h.pkl")
    y_true  = y_map[h].iloc[split:]
    y_pred  = model.predict(X_ho)
    resid   = y_true - y_pred

    # 1) Residual histogram
    ax = axes[i,0]
    ax.hist(resid, bins=50, edgecolor='k')
    ax.set_title(f"{h}h Residuals")
    ax.set_xlabel("Error (True – Pred)")
    
    # 2) Actual vs Pred scatter
    ax = axes[i,1]
    ax.scatter(y_true, y_pred, alpha=0.3)
    mn, mx = y_true.min(), y_true.max()
    ax.plot([mn,mx],[mn,mx],'r--')
    ax.set_title(f"{h}h Actual vs Pred")
    ax.set_xlabel("True")
    ax.set_ylabel("Pred")

    # 3) Top‐10 feature importance
    ax = axes[i,2]
    fi = pd.Series(model.feature_importances_, index=X_ho.columns)
    fi.nlargest(10).sort_values().plot.barh(ax=ax)
    ax.set_title(f"{h}h Top‐10 Features")

plt.tight_layout()
plt.show()

In [ ]:
# 5. Compare against existing Control_Threshold logic
import joblib
df_full = pd.read_hdf(DATA_PATH, key='df')  # raw v2 includes actual & threshold
print("\n=== Hold‑out: Forecast vs Threshold ===")
for h in HORIZONS:
    # align actual & threshold for H‑hour ahead
    actual = df_full['OnLine_Load_MW'].shift(-h).dropna().iloc[split:]
    threshold = df_full['Control_Threshold_MW'].shift(-h).dropna().iloc[split:]
    # load your saved model and predict on the same hold‑out X
    model = joblib.load(f"xgb_v2_{h}h.pkl")
    y_pred = model.predict(X.iloc[split:])
    # compute MAEs
    mae_f = mean_absolute_error(actual, y_pred)
    mae_t = mean_absolute_error(actual, threshold)
    imp  = (mae_t - mae_f) / mae_t * 100
    print(f"{h}h — MAE(threshold)={mae_t:.2f}, MAE(forecast)={mae_f:.2f}, Δ={imp:.1f}%")